# Gridworld with zero reward for each step
In this Jupyter Notebook I will rebuild and analyze the gridworld example from https://github.com/dennybritz/reinforcement-learning assigning zero reward instead of -1 for each step, and assignin a positive reward when ending on one of the two terminal states

In [ ]:
# Here I import my custom gridworld environment that I have defined in gridworld_zero_reward.py
import numpy as np
from gridworld_zero_reward import GridworldEnv_zero_reward

I generate the 4x4 gridworld enviroment with zero reward for each step

In [2]:
env_4 = GridworldEnv_zero_reward([4,4])

Now I define the policy evaluation and improvement functions to execute the policy iteration

In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        diff = 0
        for s in range(env.nS):
            vn=0
            for a, action_prob in enumerate(policy[s]):
                for prob, prox_stato, ricompensa, term in env.P[s][a]:
                    vn += action_prob * prob * (ricompensa + discount_factor * V[prox_stato])
            diff = max(diff, np.abs(vn - V[s]))
            V[s] = vn       
        if diff < theta:
            break
    return np.array(V)

In the policy improvement algorithm I had to insert a check to verify whether the value for each action in a state is equal, in that case the updated policy choose the same action as the previous one.

In [4]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Iteration Algorithm (improvement sarebbe il greedy). Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA

    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    policy_stable = True
    
    while True:
        V = policy_eval_fn(policy, env, discount_factor)
        policy_stable = True
        for s in range(env.nS):
            chosen_a = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V) 
            best_a = np.argmax(action_values) 
            # Now I check if the action values are all the same, in this case I keep the same action as before 
            if np.unique(action_values).size == 1:
                best_a = chosen_a
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
        if policy_stable:
            return policy, V
    return policy, np.zeros(env.nS)

Here i test the policy iteration with discount factor = 0.9

In [5]:
policy_09, v_09 = policy_improvement(env_4, discount_factor=0.9)
print("Policy Probability Distribution:")
print(policy_09)
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]



Here I display the value function and the policy on the grid

In [6]:
print("Reshaped Grid Value Function:")
print(v_09.reshape(env_4.shape))
print("")

Reshaped Grid Value Function:
[[  0. 100.  90.  81.]
 [100.  90.  81.  90.]
 [ 90.  81.  90. 100.]
 [ 81.  90. 100.   0.]]



In [7]:
print("Reshaped Grid Policy:")
policy_grid=str(np.reshape(np.argmax(policy_09, axis=1), env_4.shape))
policy_grid=policy_grid.replace('0', '↑')
policy_grid=policy_grid.replace('1', '→')
policy_grid=policy_grid.replace('2', '↓')
policy_grid=policy_grid.replace('3', '←')
print(policy_grid)

Reshaped Grid Policy:
[[↑ ← ← ↓]
 [↑ ↑ ↓ ↓]
 [↑ → → ↓]
 [↑ → → ↑]]


Now i test the policy improvement method imposing the discount factor equal to 1

In [8]:
env_4=GridworldEnv_zero_reward([4,4])
policy_1, v_1 = policy_improvement(env_4, discount_factor=1)
print("Policy Probability Distribution:")
print(policy_1)
print("")


Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]



In [9]:
print("Reshaped Grid Value Function:")
print(v_1.reshape(env_4.shape))
print("")

Reshaped Grid Value Function:
[[  0. 100. 100. 100.]
 [100. 100. 100. 100.]
 [100. 100. 100. 100.]
 [100. 100. 100.   0.]]



In [10]:
print("Reshaped Grid Policy:")
policy_grid=str(np.reshape(np.argmax(policy_1, axis=1), env_4.shape))
policy_grid=policy_grid.replace('0', '↑')
policy_grid=policy_grid.replace('1', '→')
policy_grid=policy_grid.replace('2', '↓')
policy_grid=policy_grid.replace('3', '←')
print(policy_grid)

Reshaped Grid Policy:
[[↑ ← ← ↓]
 [↑ ↑ ↓ ↓]
 [↑ → → ↓]
 [→ → → ↑]]


The results are slightly different but the policy is optimal in both cases

Now I consider a larger 10x10 gridworld

In [11]:
env_10 = GridworldEnv_zero_reward([10,10])

In [12]:
policy_10, v_10 = policy_improvement(env_10, discount_factor=1)
print("Policy Probability Distribution:")
print(policy_10)
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1.

In [13]:
print("Reshaped Grid Policy:")
policy_grid=str(np.reshape(np.argmax(policy_10, axis=1), env_10.shape))
policy_grid=policy_grid.replace('0', '↑')
policy_grid=policy_grid.replace('1', '→')
policy_grid=policy_grid.replace('2', '↓')
policy_grid=policy_grid.replace('3', '←')
print(policy_grid)

Reshaped Grid Policy:
[[↑ ← ← ← ← ← ← ← ← ↓]
 [↑ ↑ ← ← ← ← ← ← ↓ ↓]
 [↑ ↑ ↑ ← ← ← ← ↓ ↓ ↓]
 [↑ ↑ ↑ ↑ ← ← ↓ ↓ ↓ ↓]
 [↑ ↑ ↑ ↑ ↑ ↓ ↓ ↓ ↓ ↓]
 [↑ ↑ ↑ ↑ → → ↓ ↓ ↓ ↓]
 [↑ ↑ ↑ → → → → ↓ ↓ ↓]
 [↑ ↑ → → → → → → ↓ ↓]
 [↑ → → → → → → → → ↓]
 [→ → → → → → → → → ↑]]


In [14]:
print("Reshaped Grid Value Function:")
print(v_10.reshape(env_10.shape))
print("")

Reshaped Grid Value Function:
[[  0. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100. 100. 100. 100.   0.]]



Now I want to try a rectangular shaped gridowrld

In [15]:
env_16x9 = GridworldEnv_zero_reward([16,9])

In [16]:
policy_16x9, v_16x9 = policy_improvement(env_16x9, discount_factor=1)
print("Policy Probability Distribution:")
print(policy_16x9)
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0.

In [17]:
print("Reshaped Grid Policy:")
policy_grid=str(np.reshape(np.argmax(policy_16x9, axis=1), env_16x9.shape))
policy_grid=policy_grid.replace('0', '↑')
policy_grid=policy_grid.replace('1', '→')
policy_grid=policy_grid.replace('2', '↓')
policy_grid=policy_grid.replace('3', '←')
print(policy_grid)

Reshaped Grid Policy:
[[↑ ← ← ← ← ← ← ← ←]
 [↑ ← ← ← ← ← ← ← ←]
 [↑ ↑ ← ← ← ← ← ← ←]
 [↑ ↑ ↑ ← ← ← ← ← ←]
 [↑ ↑ ↑ ↑ ← ← ← ← ←]
 [↑ ↑ ↑ ↑ ↑ ← ← ← ↓]
 [↑ ↑ ↑ ↑ ↑ ← ↓ ↓ ↓]
 [↑ ↑ ↑ ↑ ↑ ↓ ↓ ↓ ↓]
 [↑ ↑ ↑ → ↓ ↓ ↓ ↓ ↓]
 [↑ → → → ↓ ↓ ↓ ↓ ↓]
 [→ → → → ↓ ↓ ↓ ↓ ↓]
 [→ → → → → ↓ ↓ ↓ ↓]
 [→ → → → → → ↓ ↓ ↓]
 [→ → → → → → → ↓ ↓]
 [→ → → → → → → → ↓]
 [→ → → → → → → → ↑]]


Finnaly, lets try with a very large gridworld (30x30)

In [18]:
env_30x30 = GridworldEnv_zero_reward([30,30])

In [19]:
policy_30x30, v_30x30= policy_improvement(env_30x30, discount_factor=1)

In [20]:
print("Reshaped Grid Policy:")
policy_grid=str(np.reshape(np.argmax(policy_30x30, axis=1), v_30x30.shape))
policy_grid=policy_grid.replace('0', '↑')
policy_grid=policy_grid.replace('1', '→')
policy_grid=policy_grid.replace('2', '↓')
policy_grid=policy_grid.replace('3', '←')
print(policy_grid)

Reshaped Grid Policy:
[↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ↓ ↑ ↑ ← ← ← ← ←
 ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ← ←
 ← ← ← ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ←
 ← ← ← ← ← ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓
 ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑
 ↑ ↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ←
 ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ← ←
 ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓
 ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑
 ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑
 ↑ ↑ ↑ ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ↓
 ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓
 ↓ ↓ ↓ ↓ ↓ ↓ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ → → ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↑
 ↑ 

As we can see from the examples shown above,the policy returned for this modified version of the traditional gridworld is still the best in every case. This behavior is expected if we set the discount factor less than 1, because the factor reduces the value of the future states. If instead the discount factor is set to 1 this behaviour is quite unexpected.